In [1]:
import pandas as pd
import pymysql

conn = None
cur = None

sql = ""

conn = pymysql.connect(host='172.21.27.208', user='root', password='Rkdqnrlte1q', db='Samsung_LTE', charset='utf8')
cur = conn.cursor()

sql = "SELECT * FROM site_info_ru30"
result = pd.read_sql_query(sql,conn)
info = pd.DataFrame(result)

In [2]:
while True:
    start_date = input("날짜를 입력하세요. (ex.20220101)")
    if len(start_date) == 8:
        break
    else:
        print("형식에 맞게 다시 입력해주세요. (ex.20220101)")

while True:
    finish_date = input("끝 날짜를 입력하세요. (ex.20220101)")
    if len(finish_date) == 8:
        break
    else:
        print("형식에 맞게 다시 입력해주세요. (ex.20220101)")

conn = None
cur = None

sql = ""

conn = pymysql.connect(host='172.21.27.208', user='root', password='Rkdqnrlte1q', db='Samsung_LTE', charset='utf8')
cur = conn.cursor()

new_kpi = pd.DataFrame()
for date in pd.date_range(start_date, finish_date):
    input_date = date.strftime('%Y%m%d')
    print(input_date)


#     sql = """SELECT c.sday,c.h,c.lsm, c.enb, c.cell, s.rrhid,s.rrhname,s.bw AS BW,s.ch, c.prbreal,c.rrc_total,s.pci AS PCI,s.catype,s.team,s.du_fa_num,s.maxcc,s.coloc,s.auemax,s.auemaxH
# FROM (
# SELECT a.sday,left(a.stime,2) AS h,a.lsm,a.enb,a.cell, IFNULL(SUM(a.prb_dl_total)/ SUM(a.prb_dl_cnt),0) AS prbreal, SUM(a.rrc_attempt) AS rrc_total
# FROM Samsung_LTE.kpi_""" + input_date + """ AS a
# WHERE a.stime >= '0900' AND a.stime < '2300'
# GROUP BY a.lsm,a.enb,a.cell,h
# ) AS c
# LEFT JOIN Samsung_LTE.site_info AS s ON (c.enb = s.enbname) AND (c.cell = s.cell OR c.cell = s.cell2)"""


#     sql = """SELECT c.sday,c.h,c.lsm, c.enb, c.cell, s.RRH_ID,s.RRH_Name,s.BW,s.type_MCMC, c.prbreal,c.rrc_total,s.PCI,s.ca as catype ,s.team,s.ca_info as du_fa_num ,s.max_cc_num as maxcc,s.ca_coloc as coloc,concat(c.enb,'_',s.PCI)

# FROM (

# SELECT a.sday,left(a.stime,2) AS h,a.lsm,a.enb,a.cell, IFNULL(SUM(a.prb_dl_total)/ SUM(a.prb_dl_cnt),0) AS prbreal, SUM(a.rrc_attempt) AS rrc_total

# FROM Samsung_LTE.kpi_""" + input_date + """ AS a

# WHERE a.stime >= '0900' AND a.stime < '2400'

# GROUP BY a.lsm,a.enb,a.cell,h

# ) AS c

# LEFT JOIN Samsung_LTE.site_info_ru30 AS s ON (c.enb = s.eNB_Name) AND (c.cell = s.Cell)"""

    sql = """SELECT c.sday,c.h,c.lsm, c.enb, c.cell, c.prbreal, c.rrc_total

FROM (

SELECT a.sday,left(a.stime,2) AS h,a.lsm,a.enb,a.cell, IFNULL(SUM(a.prb_dl_total)/ SUM(a.prb_dl_cnt),0) AS prbreal, SUM(a.rrc_attempt) AS rrc_total

FROM Samsung_LTE.kpi_""" + input_date + """ AS a

WHERE a.stime >= '0900' AND a.stime < '2400'

GROUP BY a.lsm,a.enb,a.cell,h

) AS c"""

    result = pd.read_sql_query(sql,conn)
    kpi = pd.DataFrame(result)

    new_kpi = pd.concat([new_kpi, kpi])

print(new_kpi)

20220203
              sday   h   lsm         enb cell    prbreal  rrc_total
0       2022-02-03  09  LSM1  SEC0006169    0  25.421591     2130.0
1       2022-02-03  10  LSM1  SEC0006169    0  12.686273     2383.0
2       2022-02-03  11  LSM1  SEC0006169    0  21.233969     2338.0
3       2022-02-03  12  LSM1  SEC0006169    0  33.003015     2891.0
4       2022-02-03  13  LSM1  SEC0006169    0  21.090768     3111.0
...            ...  ..   ...         ...  ...        ...        ...
333235  2022-02-03  19  LSM9  SEC0010746    9  36.944040     1478.0
333236  2022-02-03  20  LSM9  SEC0010746    9  39.747696     1914.0
333237  2022-02-03  21  LSM9  SEC0010746    9  68.292319     2312.0
333238  2022-02-03  22  LSM9  SEC0010746    9  32.605021     2496.0
333239  2022-02-03  23  LSM9  SEC0010746    9  53.930513     2755.0

[333240 rows x 7 columns]


In [45]:
join_df = pd.merge(info, new_kpi, left_on = ['eNB_Name', 'Cell'], right_on=['enb', 'cell'], how='inner')
join_df['enb_PCI'] = join_df['enb'] + "_" + join_df['PCI']

1.8G    243705
900M    143550
1.8H     98100
2.1E     42810
1.8E      4020
2.1G      1620
Name: BW, dtype: int64

In [4]:
ru_df = info
ru_df['enb_PCI'] = ru_df['eNB_Name'] + '_' + ru_df['PCI']
ru_df = ru_df[ru_df['BW']=='900M']
ru_df = ru_df.groupby('enb_PCI').count().reset_index()
ru_df['ru'] = ru_df['lsms']
ru_count_df = ru_df[['enb_PCI', 'ru']]
ru_count_df

,enb_PCI,ru
0,SEC0005831_-,1
1,SEC0005833_-,1
2,SEC0005834_-,2
3,SEC0005835_-,4
4,SEC0005861_117,2
...,...,...
6261,SEC9000000_81,1
6262,SECTest_302,1
6263,SECTest_329,1
6264,SECTest_420,1


In [47]:
prepro_df = join_df.pivot_table(['prbreal'], index=['sday', 'h', 'enb_PCI','BW'])
prepro_df = prepro_df.reset_index()
prepro_df

,sday,h,enb_PCI,BW,prbreal
0,2022-02-03,09,SEC0005861_117,1.8G,23.129829
1,2022-02-03,09,SEC0005861_117,1.8H,24.416188
2,2022-02-03,09,SEC0005861_117,2.1E,26.300227
3,2022-02-03,09,SEC0005861_117,900M,26.303770
4,2022-02-03,09,SEC0005861_121,1.8G,18.217197
...,...,...,...,...,...
331435,2022-02-03,23,SECTest_302,900M,0.000000
331436,2022-02-03,23,SECTest_304,1.8G,0.000000
331437,2022-02-03,23,SECTest_329,900M,0.000000
331438,2022-02-03,23,SECTest_420,900M,0.000000


In [74]:
prepro_df_900m = prepro_df[prepro_df['BW']=='900M']
prepro_df_900m = prepro_df_900m[prepro_df_900m['prbreal'] > 70]
grouped = prepro_df_900m.groupby(['enb_PCI', 'BW']).count().reset_index()
grouped2 = prepro_df.groupby(['enb_PCI', 'BW']).max().reset_index()
rrc_sum = pd.pivot_table(join_df, index=['h', 'enb_PCI', 'BW'],values='rrc_total').reset_index()
grouped3 = rrc_sum.groupby(['enb_PCI', 'BW']).sum().reset_index()
grouped['count'] = grouped['prbreal']
grouped2['max_prb'] =  grouped2['prbreal']
prb_df = grouped2[['enb_PCI', 'BW', 'max_prb']]
count_df  = grouped[['enb_PCI', 'BW', 'count']]
rrc_df = grouped3[['enb_PCI', 'BW', 'rrc_total']]
rrc_df


,enb_PCI,BW,rrc_total
0,SEC0005861_117,1.8G,22654.0
1,SEC0005861_117,1.8H,55630.0
2,SEC0005861_117,2.1E,62196.0
3,SEC0005861_117,900M,50443.0
4,SEC0005861_121,1.8G,63985.0
...,...,...,...
22091,SECTest_302,900M,0.0
22092,SECTest_304,1.8G,0.0
22093,SECTest_329,900M,0.0
22094,SECTest_420,900M,0.0


In [75]:
prepro = pd.pivot_table(prb_df, index=['enb_PCI'], columns='BW', values='max_prb').reset_index()
rrc_prepro = pd.pivot_table(rrc_df, index=['enb_PCI'], columns='BW', values='rrc_total').reset_index()
prepro.dropna(subset=['900M'], inplace=True)

In [76]:
#join_df2 = join_df[join_df['prbreal'] > 70]
prepro = prepro[prepro['900M'] > 70]

In [77]:
prepro.fillna(0, inplace=True)
rrc_prepro.fillna(0, inplace=True)

In [78]:
tmp = prepro[['enb_PCI', '900M', '1.8G', '1.8H', '1.8E', '2.1G', '2.1E']]

In [79]:
count_df = count_df[count_df['BW']=='900M']

In [80]:
count_df

,enb_PCI,BW,count
0,SEC0005861_81,900M,1
1,SEC0005862_73,900M,1
2,SEC0005865_388,900M,2
3,SEC0005872_442,900M,3
4,SEC0005873_220,900M,1
...,...,...,...
524,SEC0010777_132,900M,2
525,SEC0010777_440,900M,2
526,SEC0010787_269,900M,2
527,SEC0440009_263,900M,4


In [81]:
prepro_count = pd.merge(tmp, count_df[['enb_PCI', 'count']], left_on='enb_PCI', right_on='enb_PCI', how='inner')
result_count = pd.merge(prepro_count, rrc_prepro, left_on='enb_PCI', right_on='enb_PCI', how='inner')
result_sum = result_count['900M_y'] + result_count['1.8G_y'] + result_count['1.8E_y'] + result_count['1.8H_y'] +  result_count['2.1G_y'] + result_count['2.1E_y']

In [82]:
result_count

,enb_PCI,900M_x,1.8G_x,1.8H_x,1.8E_x,2.1G_x,2.1E_x,count,1.8E_y,1.8G_y,1.8H_y,2.1E_y,2.1G_y,900M_y
0,SEC0005861_81,73.709189,81.268848,65.757568,0.0,0.0,64.634566,1,0.0,147454.0,160190.0,75724.0,0.0,46633.0
1,SEC0005862_73,71.622191,43.170797,36.606742,0.0,0.0,39.917625,1,0.0,53759.0,83531.0,43820.0,0.0,44273.0
2,SEC0005865_388,74.020184,55.304324,33.656059,0.0,0.0,32.874125,2,0.0,90121.0,58637.0,37077.0,0.0,64233.0
3,SEC0005872_442,73.378263,84.852120,66.385064,0.0,0.0,64.409403,3,0.0,129036.0,100252.0,105766.0,0.0,60407.0
4,SEC0005873_220,77.181238,66.603584,76.946544,0.0,0.0,54.851081,1,0.0,57233.0,66578.0,39857.0,0.0,29825.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,SEC0010777_132,75.133641,0.000000,0.000000,0.0,0.0,0.000000,2,0.0,0.0,0.0,0.0,0.0,20704.0
525,SEC0010777_440,97.285064,0.000000,0.000000,0.0,0.0,0.000000,2,0.0,0.0,0.0,0.0,0.0,15044.0
526,SEC0010787_269,79.458394,58.673842,0.000000,0.0,0.0,55.866714,2,0.0,115515.0,0.0,49138.0,0.0,100725.0
527,SEC0440009_263,84.776145,79.222491,0.000000,0.0,0.0,0.000000,4,0.0,97502.0,0.0,0.0,0.0,23900.0


In [83]:
col_num = 0
for column_name in result_count:
    if 'x' in column_name:
        new_col_name = column_name[:-2] + '_max_prb'
        result_count[new_col_name] = round(result_count[column_name], 1)
        result_count.drop(column_name, axis=1, inplace=True)

    if 'y' in column_name:
        new_col_name = column_name[:-2] + '_rrc_rate'
        print(result_count[column_name].value_counts())

        result_count[new_col_name] = round((result_count[column_name] / (result_sum)) * 100)
        result_count.fillna(0, inplace=True)
        result_count[new_col_name] = result_count[new_col_name].astype(int)
        result_count[new_col_name] = result_count[new_col_name].astype(str) + '%'
        result_count.drop(column_name, axis=1, inplace=True)
    col_num += 1


0.0    529
Name: 1.8E_y, dtype: int64
0.0         40
147454.0     1
243313.0     1
68224.0      1
96958.0      1
            ..
174383.0     1
228302.0     1
166162.0     1
181197.0     1
97502.0      1
Name: 1.8G_y, Length: 490, dtype: int64
0.0         320
58637.0       2
160190.0      1
65427.0       1
69537.0       1
           ... 
62968.0       1
73265.0       1
74691.0       1
50558.0       1
15248.0       1
Name: 1.8H_y, Length: 209, dtype: int64
0.0         412
75724.0       1
136605.0      1
31706.0       1
34903.0       1
           ... 
81048.0       1
18554.0       1
39561.0       1
99142.0       1
49138.0       1
Name: 2.1E_y, Length: 118, dtype: int64
0.0        521
31190.0      1
73233.0      1
97391.0      1
29573.0      1
71471.0      1
70534.0      1
47564.0      1
30147.0      1
Name: 2.1G_y, dtype: int64
40756.0    2
54030.0    2
56532.0    2
9507.0     1
11431.0    1
          ..
42045.0    1
49084.0    1
55255.0    1
53031.0    1
5983.0     1
Name: 900M_y, Length

In [84]:
result_count

,enb_PCI,count,900M_max_prb,1.8G_max_prb,1.8H_max_prb,1.8E_max_prb,2.1G_max_prb,2.1E_max_prb,1.8E_rrc_rate,1.8G_rrc_rate,1.8H_rrc_rate,2.1E_rrc_rate,2.1G_rrc_rate,900M_rrc_rate
0,SEC0005861_81,1,73.7,81.3,65.8,0.0,0.0,64.6,0%,34%,37%,18%,0%,11%
1,SEC0005862_73,1,71.6,43.2,36.6,0.0,0.0,39.9,0%,24%,37%,19%,0%,20%
2,SEC0005865_388,2,74.0,55.3,33.7,0.0,0.0,32.9,0%,36%,23%,15%,0%,26%
3,SEC0005872_442,3,73.4,84.9,66.4,0.0,0.0,64.4,0%,33%,25%,27%,0%,15%
4,SEC0005873_220,1,77.2,66.6,76.9,0.0,0.0,54.9,0%,30%,34%,21%,0%,15%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,SEC0010777_132,2,75.1,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,100%
525,SEC0010777_440,2,97.3,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,100%
526,SEC0010787_269,2,79.5,58.7,0.0,0.0,0.0,55.9,0%,44%,0%,19%,0%,38%
527,SEC0440009_263,4,84.8,79.2,0.0,0.0,0.0,0.0,0%,80%,0%,0%,0%,20%


In [89]:
tmp_list = result_count[['900M_max_prb', '1.8G_max_prb', '1.8H_max_prb', '1.8E_max_prb', '2.1E_max_prb', '2.1G_max_prb']]
count_list = []
for i, v  in tmp_list.iterrows():
    count = 0
    for a in v:
        if a != 0:
            count += 1
    count_list.append(count)
result_count['fa'] = count_list
result_count


,enb_PCI,count,900M_max_prb,1.8G_max_prb,1.8H_max_prb,1.8E_max_prb,2.1G_max_prb,2.1E_max_prb,1.8E_rrc_rate,1.8G_rrc_rate,1.8H_rrc_rate,2.1E_rrc_rate,2.1G_rrc_rate,900M_rrc_rate,fa
0,SEC0005861_81,1,73.7,81.3,65.8,0.0,0.0,64.6,0%,34%,37%,18%,0%,11%,4
1,SEC0005862_73,1,71.6,43.2,36.6,0.0,0.0,39.9,0%,24%,37%,19%,0%,20%,4
2,SEC0005865_388,2,74.0,55.3,33.7,0.0,0.0,32.9,0%,36%,23%,15%,0%,26%,4
3,SEC0005872_442,3,73.4,84.9,66.4,0.0,0.0,64.4,0%,33%,25%,27%,0%,15%,4
4,SEC0005873_220,1,77.2,66.6,76.9,0.0,0.0,54.9,0%,30%,34%,21%,0%,15%,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,SEC0010777_132,2,75.1,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,100%,1
525,SEC0010777_440,2,97.3,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,100%,1
526,SEC0010787_269,2,79.5,58.7,0.0,0.0,0.0,55.9,0%,44%,0%,19%,0%,38%,3
527,SEC0440009_263,4,84.8,79.2,0.0,0.0,0.0,0.0,0%,80%,0%,0%,0%,20%,2


In [90]:
prepro_df2 = join_df.pivot_table(['prbreal'], index=['enb','PCI','BW', 'lsm', 'RS_POWER', 'type_MCMC'])
prepro_df2 = prepro_df2.reset_index()
prepro_df2['enb_PCI'] = prepro_df2['enb'] + "_" + prepro_df2['PCI']
prepro_df_900m = prepro_df2[prepro_df2['BW']=='900M']
prepro_df_900m

,enb,PCI,BW,lsm,RS_POWER,type_MCMC,prbreal,enb_PCI
3,SEC0005861,117,900M,LSM5,ci_pwrOpt_Same,MCMC,40.282447,SEC0005861_117
7,SEC0005861,121,900M,LSM5,ci_pwrOpt_Same,MCMC,36.579671,SEC0005861_121
11,SEC0005861,314,900M,LSM5,ci_pwrOpt_Same,MCMC,43.064323,SEC0005861_314
15,SEC0005861,41,900M,LSM5,ci_pwrOpt_Same,MCMC,34.798855,SEC0005861_41
18,SEC0005861,442,900M,LSM5,ci_pwrOpt_Same,MCMC,32.862341,SEC0005861_442
...,...,...,...,...,...,...,...,...
22095,SEC9000000,81,900M,LSM10,ci_pwrOpt_Same,MCMC,37.001599,SEC9000000_81
22097,SECTest,302,900M,LSM11,ci_pwrOpt_OneHalf,MCMC,0.000000,SECTest_302
22099,SECTest,329,900M,LSM11,ci_pwrOpt_Half,MCMC,0.000000,SECTest_329
22100,SECTest,420,900M,LSM11,ci_pwrOpt_Half,MCMC,0.000000,SECTest_420


In [91]:
results = pd.merge(result_count, prepro_df_900m[['enb_PCI', 'lsm', 'RS_POWER', 'type_MCMC']], left_on='enb_PCI', right_on='enb_PCI', how='inner')
results

,enb_PCI,count,900M_max_prb,1.8G_max_prb,1.8H_max_prb,1.8E_max_prb,2.1G_max_prb,2.1E_max_prb,1.8E_rrc_rate,1.8G_rrc_rate,1.8H_rrc_rate,2.1E_rrc_rate,2.1G_rrc_rate,900M_rrc_rate,fa,lsm,RS_POWER,type_MCMC
0,SEC0005861_81,1,73.7,81.3,65.8,0.0,0.0,64.6,0%,34%,37%,18%,0%,11%,4,LSM5,ci_pwrOpt_Same,MCMC
1,SEC0005862_73,1,71.6,43.2,36.6,0.0,0.0,39.9,0%,24%,37%,19%,0%,20%,4,LSM5,ci_pwrOpt_Same,MCMC
2,SEC0005865_388,2,74.0,55.3,33.7,0.0,0.0,32.9,0%,36%,23%,15%,0%,26%,4,LSM5,ci_pwrOpt_Same,MCMC
3,SEC0005872_442,3,73.4,84.9,66.4,0.0,0.0,64.4,0%,33%,25%,27%,0%,15%,4,LSM7,ci_pwrOpt_Half,MCMC
4,SEC0005873_220,1,77.2,66.6,76.9,0.0,0.0,54.9,0%,30%,34%,21%,0%,15%,4,LSM7,ci_pwrOpt_Same,MCMC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,SEC0010777_132,2,75.1,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,100%,1,LSM10,ci_pwrOpt_Half,MCMC
525,SEC0010777_440,2,97.3,0.0,0.0,0.0,0.0,0.0,0%,0%,0%,0%,0%,100%,1,LSM10,ci_pwrOpt_Half,MCMC
526,SEC0010787_269,2,79.5,58.7,0.0,0.0,0.0,55.9,0%,44%,0%,19%,0%,38%,3,LSM3,ci_pwrOpt_Half,MCMC
527,SEC0440009_263,4,84.8,79.2,0.0,0.0,0.0,0.0,0%,80%,0%,0%,0%,20%,2,LSM2,ci_pwrOpt_Same,MCMC


In [105]:
real_result =  pd.merge(results, ru_count_df, left_on='enb_PCI', right_on='enb_PCI', how='inner')
real_result
real_result = real_result[['lsm', 'enb_PCI', '900M_max_prb', '1.8G_max_prb', '1.8H_max_prb', '1.8E_max_prb', '2.1E_max_prb', '2.1G_max_prb', 'fa',
       '900M_rrc_rate', '1.8G_rrc_rate', '1.8H_rrc_rate', '1.8E_rrc_rate', '2.1E_rrc_rate', '2.1G_rrc_rate', 'type_MCMC', 'ru', 'RS_POWER', 'count']]
real_result

,lsm,enb_PCI,900M_max_prb,1.8G_max_prb,1.8H_max_prb,1.8E_max_prb,2.1E_max_prb,2.1G_max_prb,fa,900M_rrc_rate,1.8G_rrc_rate,1.8H_rrc_rate,1.8E_rrc_rate,2.1E_rrc_rate,2.1G_rrc_rate,type_MCMC,ru,RS_POWER,count
0,LSM5,SEC0005861_81,73.7,81.3,65.8,0.0,64.6,0.0,4,11%,34%,37%,0%,18%,0%,MCMC,2,ci_pwrOpt_Same,1
1,LSM5,SEC0005862_73,71.6,43.2,36.6,0.0,39.9,0.0,4,20%,24%,37%,0%,19%,0%,MCMC,2,ci_pwrOpt_Same,1
2,LSM5,SEC0005865_388,74.0,55.3,33.7,0.0,32.9,0.0,4,26%,36%,23%,0%,15%,0%,MCMC,2,ci_pwrOpt_Same,2
3,LSM7,SEC0005872_442,73.4,84.9,66.4,0.0,64.4,0.0,4,15%,33%,25%,0%,27%,0%,MCMC,2,ci_pwrOpt_Half,3
4,LSM7,SEC0005873_220,77.2,66.6,76.9,0.0,54.9,0.0,4,15%,30%,34%,0%,21%,0%,MCMC,2,ci_pwrOpt_Same,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,LSM10,SEC0010777_132,75.1,0.0,0.0,0.0,0.0,0.0,1,100%,0%,0%,0%,0%,0%,MCMC,1,ci_pwrOpt_Half,2
525,LSM10,SEC0010777_440,97.3,0.0,0.0,0.0,0.0,0.0,1,100%,0%,0%,0%,0%,0%,MCMC,1,ci_pwrOpt_Half,2
526,LSM3,SEC0010787_269,79.5,58.7,0.0,0.0,55.9,0.0,3,38%,44%,0%,0%,19%,0%,MCMC,2,ci_pwrOpt_Half,2
527,LSM2,SEC0440009_263,84.8,79.2,0.0,0.0,0.0,0.0,2,20%,80%,0%,0%,0%,0%,MCMC,1,ci_pwrOpt_Same,4


In [107]:
real_result.to_csv('./prb.csv', header=True, index=False)